In [1]:
from datastreams.datastream import Streamer

import matplotlib.pyplot as plt
import pandas as pd
import polars as pl

# These commands enlarge the column size of the dataframe so things like 0x... are not truncated
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [2]:
# DEFINE TIMESTAMP HERE. Timstamp is used for replication quality assurance purposes.
timestamp = 1700000000

# define ethereum token addresses here to be used in cowswap trades query filter
weth_addr = "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2"
usdc_addr = "0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48"

# we set a fixed query size number. The Cow settlements and Uniswap swaps query are multiples larger than this initial query size.
query_size = 250

#Filter size - We filter trades out that are smaller than $1000 USD size
filter_usd = 10000

In [3]:
weth_usdc_list = [
    "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640", # usdc/weth .05%
    "0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8" #usdc/weth .3%
]

swaps_df_list = []

In [4]:
# univ3_messari_ds = Streamer('https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-ethereum')
univ3_no_messari_ds = Streamer('https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3')

In [5]:
# get a query field path from the query dictionary which is automatically populated in the Streamer object
swaps_fp = univ3_no_messari_ds.queryDict.get('swaps')

In [6]:
# TODO - how to expose the transaction gasUsed and gasPrice to the swaps query?

In [7]:
# for loop over the LP list to get the swap data
for lp in weth_usdc_list:
    # add parameters to the query_path
    swaps_qp = swaps_fp(
        first=query_size * 5,
        orderBy='timestamp',
        orderDirection='asc',
        where = {'timestamp_lt': timestamp, 'amountUSD_gt': filter_usd, 'pool': lp} 
        )

    # run query
    swaps_df = univ3_no_messari_ds.runQuery(swaps_qp)
    swaps_df_list.append(swaps_df)

FIELD - swaps
FIELD - swaps


In [8]:
# concat swaps_df_list into a single dataframe.
swaps_df = pd.concat(swaps_df_list)

In [9]:
swaps_df

,swaps_id,swaps_transaction_id,swaps_timestamp,swaps_pool_id,swaps_token0_id,swaps_token1_id,swaps_sender,swaps_recipient,swaps_origin,swaps_amount0,swaps_amount1,swaps_amountUSD,swaps_sqrtPriceX96,swaps_tick,swaps_logIndex,endpoint
0,0x0d6056ed25185f8c957cfbbb0c13338f67e31528e7732aa4775f3f14475c5c40#232,0x0d6056ed25185f8c957cfbbb0c13338f67e31528e7732aa4775f3f14475c5c40,1620293574,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xe592427a0aece92de3edee1f18e0157c05861564,0xc521c3860299e873df9bff4b23ea25aae8ea1cb1,0xc521c3860299e873df9bff4b23ea25aae8ea1cb1,-10965.018946,3.207381,11009.463137,1359632855626749404036039642889986,195017,26,uniswap-v3
1,0xdc4a1c1cb3eb87cc58258e425e2e5f1aa73b44f821630e21156748accf174c5b#233,0xdc4a1c1cb3eb87cc58258e425e2e5f1aa73b44f821630e21156748accf174c5b,1620293752,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xe592427a0aece92de3edee1f18e0157c05861564,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,0x0197abe48f0a9826d10f44dad7050f26586eb9ba,10000.000000,-2.924054,10038.650438,1350641974378375808946729382812276,194884,77,uniswap-v3
2,0x5cf030b5573c12c0976e2e3813b2ef0bdd07dd0e7828c3a0f08a625f8374664d#467,0x5cf030b5573c12c0976e2e3813b2ef0bdd07dd0e7828c3a0f08a625f8374664d,1620314570,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xe592427a0aece92de3edee1f18e0157c05861564,0x22f9dcf4647084d6c31b2765f6910cd85c178c18,0x0c42fa4328d33338e5b646ec2da7854c26206330,11818.615984,-3.341102,11821.122580,1329434365800569818101171284175681,194568,320,uniswap-v3
3,0x641f455de6342e4c3e78d23b0be09128f70fde1827c217e8aeece19f62d84927#487,0x641f455de6342e4c3e78d23b0be09128f70fde1827c217e8aeece19f62d84927,1620316160,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xe592427a0aece92de3edee1f18e0157c05861564,0xe592427a0aece92de3edee1f18e0157c05861564,0x53a80861e45c59b67635c10b1eb800062ee5cec2,10856.594653,-3.046881,10849.032813,1324848904626658899636847484342713,194499,176,uniswap-v3
4,0x2ed45dd76d3eed7d9d5d61b933eec7938bb3e92cd9089c9e8509ee6dda9bcd77#498,0x2ed45dd76d3eed7d9d5d61b933eec7938bb3e92cd9089c9e8509ee6dda9bcd77,1620318583,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xe592427a0aece92de3edee1f18e0157c05861564,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,0x096b907590dce529dfcbafe61d3e6806267666b3,10000.000000,-2.791240,10004.640741,1321470531815907155879550743662544,194448,68,uniswap-v3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,0xed3e4ee7dc6c931d8d325232bff796dbf467ab2ca4bb53b56c270a4a112a86fa#3583,0xed3e4ee7dc6c931d8d325232bff796dbf467ab2ca4bb53b56c270a4a112a86fa,1620534441,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xe592427a0aece92de3edee1f18e0157c05861564,0xf3b37ad3c55d9f3a963535b78242cf0ccc5b4292,0xf3b37ad3c55d9f3a963535b78242cf0ccc5b4292,-10000.000000,2.604739,10015.162713,1276775026938462173104426182627573,193760,288,uniswap-v3
1246,0xfe458e42cf038adcb8e7b570beec9ee0c872d65c7b184b583da19e6146397f11#3584,0xfe458e42cf038adcb8e7b570beec9ee0c872d65c7b184b583da19e6146397f11,1620534454,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xe592427a0aece92de3edee1f18e0157c05861564,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,0x048fde2d3d47f54d3e7be0be7a40e1cb5aff5d72,68492.624238,-17.731237,68384.419842,1276570649913368666614403268940889,193756,246,uniswap-v3
1247,0x4c6024245dcd7f9a388c1ac4cd25f7e81c8442cc4a4804b1c248dc055c01c769#3585,0x4c6024245dcd7f9a388c1ac4cd25f7e81c8442cc4a4804b1c248dc055c01c769,1620534641,0x8ad599c3a0ff1de082011e

In [10]:
timestamp_list = swaps_df['swaps_timestamp'].to_list()

### Query Transaction Gas

In [11]:
txs_fp = univ3_no_messari_ds.queryDict.get('transactions')

In [12]:
txs_df_list = []

In [14]:
# for loop over the LP list to get the swap data
for lp in weth_usdc_list:
    # add parameters to the query_path
    txs_qp = txs_fp(
        first=query_size * 5,
        orderBy='timestamp',
        orderDirection='asc',
        where = {"swaps_": {'timestamp_lt': timestamp, 'amountUSD_gt': filter_usd, 'pool': lp}}
        )
    # run query
    txs_df = univ3_no_messari_ds.runQuery(txs_qp)
    txs_df_list.append(txs_df)

FIELD - transactions
FIELD - transactions


In [15]:
# concat swaps_df_list into a single dataframe.
txs_full_df = pd.concat(txs_df_list)

In [16]:
txs_full_df

,transactions_id,transactions_blockNumber,transactions_timestamp,transactions_gasUsed,transactions_gasPrice,endpoint
0,0x0d6056ed25185f8c957cfbbb0c13338f67e31528e7732aa4775f3f14475c5c40,12379943,1620293574,357772,80000000000,uniswap-v3
1,0xdc4a1c1cb3eb87cc58258e425e2e5f1aa73b44f821630e21156748accf174c5b,12379952,1620293752,594343,63000100001,uniswap-v3
2,0x5cf030b5573c12c0976e2e3813b2ef0bdd07dd0e7828c3a0f08a625f8374664d,12381496,1620314570,455000,79000000000,uniswap-v3
3,0x641f455de6342e4c3e78d23b0be09128f70fde1827c217e8aeece19f62d84927,12381630,1620316160,266356,76000000000,uniswap-v3
4,0x2ed45dd76d3eed7d9d5d61b933eec7938bb3e92cd9089c9e8509ee6dda9bcd77,12381798,1620318583,672868,133228551344,uniswap-v3
...,...,...,...,...,...,...
1245,0xed3e4ee7dc6c931d8d325232bff796dbf467ab2ca4bb53b56c270a4a112a86fa,12397992,1620534441,229771,250000000000,uniswap-v3
1246,0xfe458e42cf038adcb8e7b570beec9ee0c872d65c7b184b583da19e6146397f11,12397995,1620534454,753781,279431468402,uniswap-v3
1247,0x4c6024245dcd7f9a388c1ac4cd25f7e81c8442cc4a4804b1c248dc055c01c769,12398009,1620534641,270186,220000001561,uniswap-v3
1248,0xda8703922068b91acf26e7b821e5ec72b67ee056d21aeb438fbc6bef3b466bf4,12398014,1620534716,263769,220000001459,uniswap-v3


In [17]:
# join swaps_df with txs_full_df on swaps_timestamp and transactions_timestamp
merged_df = swaps_df.merge(txs_full_df, left_on='swaps_transaction_id', right_on='transactions_id', how='inner')

In [25]:
# drop duplicates
merged_df = merged_df.drop_duplicates(subset=['swaps_transaction_id'])

In [26]:
merged_df

,swaps_id,swaps_transaction_id,swaps_timestamp,swaps_pool_id,swaps_token0_id,swaps_token1_id,swaps_sender,swaps_recipient,swaps_origin,swaps_amount0,swaps_amount1,swaps_amountUSD,swaps_sqrtPriceX96,swaps_tick,swaps_logIndex,endpoint_x,transactions_id,transactions_blockNumber,transactions_timestamp,transactions_gasUsed,transactions_gasPrice,endpoint_y
0,0x0d6056ed25185f8c957cfbbb0c13338f67e31528e7732aa4775f3f14475c5c40#232,0x0d6056ed25185f8c957cfbbb0c13338f67e31528e7732aa4775f3f14475c5c40,1620293574,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xe592427a0aece92de3edee1f18e0157c05861564,0xc521c3860299e873df9bff4b23ea25aae8ea1cb1,0xc521c3860299e873df9bff4b23ea25aae8ea1cb1,-10965.018946,3.207381,11009.463137,1359632855626749404036039642889986,195017,26,uniswap-v3,0x0d6056ed25185f8c957cfbbb0c13338f67e31528e7732aa4775f3f14475c5c40,12379943,1620293574,357772,80000000000,uniswap-v3
2,0xdc4a1c1cb3eb87cc58258e425e2e5f1aa73b44f821630e21156748accf174c5b#233,0xdc4a1c1cb3eb87cc58258e425e2e5f1aa73b44f821630e21156748accf174c5b,1620293752,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xe592427a0aece92de3edee1f18e0157c05861564,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,0x0197abe48f0a9826d10f44dad7050f26586eb9ba,10000.000000,-2.924054,10038.650438,1350641974378375808946729382812276,194884,77,uniswap-v3,0xdc4a1c1cb3eb87cc58258e425e2e5f1aa73b44f821630e21156748accf174c5b,12379952,1620293752,594343,63000100001,uniswap-v3
4,0x5cf030b5573c12c0976e2e3813b2ef0bdd07dd0e7828c3a0f08a625f8374664d#467,0x5cf030b5573c12c0976e2e3813b2ef0bdd07dd0e7828c3a0f08a625f8374664d,1620314570,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xe592427a0aece92de3edee1f18e0157c05861564,0x22f9dcf4647084d6c31b2765f6910cd85c178c18,0x0c42fa4328d33338e5b646ec2da7854c26206330,11818.615984,-3.341102,11821.122580,1329434365800569818101171284175681,194568,320,uniswap-v3,0x5cf030b5573c12c0976e2e3813b2ef0bdd07dd0e7828c3a0f08a625f8374664d,12381496,1620314570,455000,79000000000,uniswap-v3
6,0x641f455de6342e4c3e78d23b0be09128f70fde1827c217e8aeece19f62d84927#487,0x641f455de6342e4c3e78d23b0be09128f70fde1827c217e8aeece19f62d84927,1620316160,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xe592427a0aece92de3edee1f18e0157c05861564,0xe592427a0aece92de3edee1f18e0157c05861564,0x53a80861e45c59b67635c10b1eb800062ee5cec2,10856.594653,-3.046881,10849.032813,1324848904626658899636847484342713,194499,176,uniswap-v3,0x641f455de6342e4c3e78d23b0be09128f70fde1827c217e8aeece19f62d84927,12381630,1620316160,266356,76000000000,uniswap-v3
8,0x2ed45dd76d3eed7d9d5d61b933eec7938bb3e92cd9089c9e8509ee6dda9bcd77#498,0x2ed45dd76d3eed7d9d5d61b933eec7938bb3e92cd9089c9e8509ee6dda9bcd77,1620318583,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xe592427a0aece92de3edee1f18e0157c05861564,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,0x096b907590dce529dfcbafe61d3e6806267666b3,10000.000000,-2.791240,10004.640741,1321470531815907155879550743662544,194448,68,uniswap-v3,0x2ed45dd76d3eed7d9d5d61b933eec7938bb3e92cd9089c9e8509ee6dda9bcd77,12381798,1620318583,672868,133228551344,uniswap-v3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3769,0xed3e4ee7dc6c931d8d325232bff796dbf467ab2ca4bb53b56c270a4a112a86fa#3583,0xed3e4ee7dc6c931d8d325232bff796dbf467ab2ca4bb53b56c270a4a112a86fa,1620534441,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xe592427a0aece92de3edee1f18e0157c05861564,0xf3b37ad3c55d9f3a963535b78242cf0ccc5b4292,0xf3b37ad3c55d9f3a963535b78242cf0ccc5b4292,-10000.000000,2.604739,10015.162713,1276775026938

In [27]:
# print min and max swaps_amountUSD
print(merged_df['swaps_amountUSD'].min())
print(merged_df['swaps_amountUSD'].max())

10000.335428718183
1136162.2839526555


In [28]:
# check the merge on timestamps was correct
sum(merged_df['swaps_timestamp'] - merged_df['transactions_timestamp'])

0

In [29]:
# check unique values in swaps_pool_id
merged_df['swaps_pool_id'].unique()

array(['0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640',
       '0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8'], dtype=object)

In [30]:
merged_df['swaps_token1_id'].unique()

array(['0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'], dtype=object)

In [31]:
merged_df['swaps_token0_id'].unique()

array(['0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'], dtype=object)

In [32]:
# print min/max swaps_timestamp
print(merged_df['swaps_timestamp'].min())
print(merged_df['swaps_timestamp'].max())

1620244502
1620659361
